In [1]:
%run uvms_dynamics.ipynb

jit after {'jit': True, 'jit_options': {'flags': '-Ofast'}, 'compiler': 'shell'}
number of joints = 4
floating_base found
floating_base found
floating_base found


In [2]:
# conditional function instances
# coupled forward dynamics of uvms
x_next_coupled, states, u, dt, q_min, q_max, parameters_coupled, base_T, states_checks, u_checks  = rig_dyn.forward_dynamics(gravity=9.81, 
                                                                                    floating_base_id=tau_F,
                                                                                    floating_base_bias_f=b_F , 
                                                                                    J_uv = J_uv, 
                                                                                    coupled=True)
x_next_coupled_eval = cs.Function('UVMSnext_coupled', [states, u, dt, parameters_coupled, base_T ,q_min, q_max], [x_next_coupled])


# decoupled forward dynamics of uvms
x_next_decoupled, states, u, dt, q_min, q_max, parameters_decoupled, base_T, states_checks, u_checks  = rig_dyn.forward_dynamics(gravity=9.81, 
                                                                                    floating_base_id=tau_F,
                                                                                    floating_base_bias_f=b_F , 
                                                                                    J_uv = J_uv, 
                                                                                    coupled=False)

x_next_decoupled_eval = cs.Function('UVMSnext_decoupled', [states, u, dt, parameters_decoupled, base_T ,q_min, q_max], [x_next_decoupled])



UVMSnext_use_coupled = cs.Function.if_else('UVMSnext_use_coupled', x_next_coupled_eval, x_next_decoupled_eval)

floating_base found
floating_base found
floating_base found
floating_base found
floating_base found
floating_base found


In [3]:
# # c , cpp or matlab code generation for forward dynamics
# UVMSnext_use_coupled.generate("x_next_eval.c")
# os.system(f"gcc -fPIC -shared x_next_eval.c -o libUVMS_xnext.so")

In [4]:
alpha.joint_min

array([0.  , 1.5 , 0.05, 0.  ])

In [5]:
alpha.joint_max

array([5.7, 3.4, 3.4, 5.7])

In [6]:
st_x0 = cs.vertcat(0,0,0, 0,0,0, 0.0, 1.5, 0.99245, 3.16073,   0, 0, 0, 0, 0, 0,  0.00751799, -0.0073626, 0.00281749, 0.0222385)
u0 = cs.vertcat(0, 0, 0, 0, 0, 0, 0.0, -1.0, 0.0, 0.0)

UVMSnext_use_coupled(True, st_x0, u0, alpha.delta_t, alpha.sim_p, alpha.base_T0, alpha.joint_min, alpha.joint_max)

DM([5.25251e-07, 2.44134e-07, 8.30547e-05, -1.66262e-05, 2.08965e-05, -1.17623e-05, 5.63917e-05, 1.5, 0.992479, 3.16098, 1.21199e-05, 5.78771e-06, -8.34566e-05, -0.00040163, 0.000496505, -0.000255055, 9.64247e-06, -1.38154e-07, 2.13895e-05, 0.000271359])

In [11]:
sim = x_next_decoupled_eval.mapaccum(alpha.N)
x0 = np.array([0, 0, 1, 0, 0, 0, 0.0, 1.5, 0.05, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) #x y z r p y q0 q1 q2 q3 u v w p q r qd0 qd1 qd2 qd3 qd4
u0 = np.array([0, 0.0, 0, 0, 0, 0, 0.1, 0, 0, 0])
res = sim(x0, u0, alpha.delta_t, alpha.sim_p,  alpha.base_T0, alpha.joint_min, alpha.joint_max)
t = np.linspace(0, alpha.sim_n, alpha.N+1)


# Create a figure and plot the data
plt.figure()
plt.plot(t, np.concatenate((np.array(x0).reshape((20,1)), res.toarray()), axis=1)[0:3,:].T)
plt.legend(['surge - x[m]', 'sway - y[m]', 'heave -z[m]'])
plt.title("position in NED coordinate for floating vehicle")
plt.xlabel('t [s]')

plt.figure()
plt.plot(t, np.concatenate((np.array(x0).reshape((20,1)), res.toarray()), axis=1)[3:6,:].T)
plt.legend(['roll', 'pitch', 'yaw'])
plt.title("orientation in NED euler coordinate")
plt.xlabel('t [s]')

plt.figure()
plt.plot(t, np.concatenate((np.array(x0).reshape((20,1)), res.toarray()), axis=1)[6:10,:].T)
plt.legend(['q0', 'q1', 'q2', 'q3'])
plt.title("positions for manipulator")
plt.xlabel('t [s]')

plt.figure()
plt.plot(t, np.concatenate((np.array(x0).reshape((20,1)), res.toarray()), axis=1)[10:13,:].T)
plt.legend(['u', 'v', 'w'])
plt.xlabel('t[s]')
plt.title("linear velocity in body coordinate")
plt.show()

plt.figure()
plt.plot(t, np.concatenate((np.array(x0).reshape((20,1)), res.toarray()), axis=1)[13:16,:].T)
plt.legend(['p', 'q', 'r'])
plt.xlabel('t[s]')
plt.title("rotational velocity in body coordinate")
plt.show()

plt.figure()
plt.plot(t, np.concatenate((np.array(x0).reshape((20,1)), res.toarray()), axis=1)[16:20,:].T)
plt.legend(['dq0', 'dq1', 'dq2', 'dq3'])
plt.xlabel('t[s]')
plt.title("linear velocity in manipulator joints")
plt.show()

SystemError: <built-in function Function_call> returned a result with an error set